In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
df = pd.read_csv('Article_top_Data.csv')
df.head()

,Unnamed: 0,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,ReceiptRowNumber,Time,ArticleNumber,ArticleName,Quantity,Timestamp,Month,Day,Hour,Group
0,4,2022-01-03,1001,1,296,5,09:21:14,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:21:14,1,0,9,Milch
1,6,2022-01-03,1001,1,296,7,09:21:14,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:21:14,1,0,9,Pfandartikel
2,7,2022-01-03,1001,1,296,8,09:21:14,435066,Schlaufentragetasche ST,1.0,2022-01-03 09:21:14,1,0,9,Schlaufentragetasche ST
3,19,2022-01-03,1001,1,300,1,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
4,20,2022-01-03,1001,1,300,2,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST


In [3]:
x = df[(df.Group=="BO-Laugenbreze 1ST")]
x = x.groupby(['Date'], as_index=False).aggregate({'Quantity': 'sum'})
x.head()

,Date,Quantity
0,2022-01-03,1005.0
1,2022-01-04,735.0
2,2022-01-05,1009.0
3,2022-01-07,887.0
4,2022-01-08,855.0


In [4]:
x.head()

,Date,Quantity
0,2022-01-03,1005.0
1,2022-01-04,735.0
2,2022-01-05,1009.0
3,2022-01-07,887.0
4,2022-01-08,855.0


In [5]:
x.shape

(303, 2)

In [6]:
x.Date = pd.to_datetime(x.Date)


In [7]:
x_train = x[x.Date.dt.month<9]
x_test = x[x.Date.dt.month>=9]

In [8]:
x_train.Quantity

0      1005.0
1       735.0
2      1009.0
3       887.0
4       855.0
        ...  
195     247.0
196     474.0
197     563.0
198     508.0
199     572.0
Name: Quantity, Length: 200, dtype: float64

In [9]:
p=1
d=1
q=1

model = ARIMA(x_train.Quantity, order=(p,d,q))

model_fit = model.fit()

forecast_steps = 103
forecast = model_fit.forecast(steps=forecast_steps)

print(forecast)

rmse = mean_squared_error(x_test.Quantity, forecast, squared=False)
print("RMSE:", rmse)


200    558.501042
201    563.611715
202    561.676827
203    562.409371
204    562.132031
          ...    
298    562.208196
299    562.208196
300    562.208196
301    562.208196
302    562.208196
Name: predicted_mean, Length: 103, dtype: float64
RMSE: 449.5715809299639


In [11]:
forecast.sum()

57904.75507192197